#### 1. Links

In [15]:
#langchain Documentation:-
#[ChatGroq]: https://python.langchain.com/v0.2/docs/integrations/chat/groq/

# LangChain Expression Language (LCEL): https://python.langchain.com/v0.1/docs/modules/model_io/chat/quick_start/

#LLM Api Documentation:-
#[Groq]: https://console.groq.com/docs/quickstart

In [16]:
# Prompt Template Docs:
#   https://python.langchain.com/v0.2/docs/concepts/#prompt-templateshttps://python.langchain.com/v0.2/docs/concepts/#prompt-templates

In [27]:
# 1.Zero-shot learning: The model receives a prompt for a task it hasn't been explicitly trained on, without providing any examples.

# 2.Few-shot learning: The model is given a few examples of how to perform a task before being asked to generalize to similar tasks.

In [17]:
%pip install -qU langchain-groq python-dotenv

You should consider upgrading via the '/Users/taurangela/Desktop/Github/Langchain-with-Groq/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [19]:
# Load environment variables from .env
load_dotenv()

# Access the GROQ_API_KEY
api_key = os.getenv('GROQ_API_KEY')

In [20]:
# Create a Groq model
llm = ChatGroq(
    model="mixtral-8x7b-32768" #choose model from the list:https://console.groq.com/docs/models
)


# Optional configuration
# llm = ChatGroq(
#     model="mixtral-8x7b-32768",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     # other params...
# )

In [21]:
# PART 1: Create a ChatPromptTemplate using a template string
print("-----Prompt from Template-----")

template = "Tell me a joke about {topic}."
prompt_template = ChatPromptTemplate.from_template(template)

prompt = prompt_template.invoke({"topic": "cats"})
result = llm.invoke(prompt)
print(result.content)

-----Prompt from Template-----
What do you call a pile of kittens? A meow-ntain! I'll show myself out.


In [22]:
# PART 1.1: Zero-Shot Learning Example
print("\n----- Zero-Shot Learning Example -----\n")

# Prompt without any prior examples; purely based on model's training
zero_shot_prompt = ChatPromptTemplate.from_template("Translate the following English sentence to French: '{sentence}'")
prompt = zero_shot_prompt.invoke({"sentence": "Hello, how are you?"})
result = llm.invoke(prompt)
print(result.content)



----- Zero-Shot Learning Example -----

The French translation of the English sentence "Hello, how are you?" is "Bonjour, comment ça va?" In French, "Bonjour" is used to say hello, and "comment ça va" is a common way to ask someone how they are doing.


In [23]:
# PART 2: Prompt with Multiple Placeholders
print("\n----- Prompt with Multiple Placeholders -----\n")

template_multiple = """You are a helpful assistant.
Human: Tell me a {adjective} short story about a {animal}.
Assistant:"""

prompt_multiple = ChatPromptTemplate.from_template(template_multiple)
prompt = prompt_multiple.invoke({"adjective": "funny", "animal": "panda"})

result = llm.invoke(prompt)
print(result.content)


----- Prompt with Multiple Placeholders -----

Sure, I'd be happy to! Here's a light-hearted story for you:

Once upon a time, in the lush bamboo forests of China, there lived a panda named Pambo. Pambo was a bit of an oddball. Unlike other pandas who were content with eating bamboo all day, Pambo had a secret love for... drum rolls please... ice cream!

One day, Pambo stumbled upon a forgotten ice cream truck at the edge of the forest. His eyes widened in delight as he saw rows and rows of his favorite flavor - vanilla! With his nimble paws, he managed to open the truck and enjoyed a feast of ice cream.

After his delightful meal, Pambo tried to push the ice cream truck back into the forest. But try as he might, the truck was just too heavy. Suddenly, he had a brilliant idea! He started to stack bamboo sticks under the truck, creating a makeshift ramp. With a final push, the truck rolled onto the ramp and into the forest with a soft "bump".

From that day forward, the forest was fill

In [24]:
# PART 3: Prompt with System and Human Messages (Using Tuples)
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")

messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]

'''
# This does NOT work:
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    HumanMessage(content="Tell me {joke_count} jokes."),
]
'''

prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
result = llm.invoke(prompt)
print(result.content)


----- Prompt with System and Human Messages (Tuple) -----

Sure, I'd be happy to share some lawyer jokes with you! Here they are:

1. Why did the lawyer bring a ladder to court?
In case he needed to raise the bar!
2. What's the difference between a lawyer and a herd of buffalo?
The lawyer charges more.
3. How does an attorney sleep?
First, they lie on one side. Then they lie on the other.

I hope these brought a smile to your face! Just remember, they're all in good fun and not meant to offend or stereotype all lawyers.


In [25]:
# PART 4: Using Conditional Logic in Prompts
print("\n----- Using Conditional Logic in Prompts -----\n")

template_conditional = """You are a helpful assistant.
Human: Should I {action} the {object}? 
Assistant: {conditional_response}"""

prompt_conditional = ChatPromptTemplate.from_template(template_conditional)

# Example with action "buy" and object "car"
prompt = prompt_conditional.invoke({
    "action": "buy",
    "object": "car",
    "conditional_response": "If you need a car and can afford it, yes, you should buy it."
})

result = llm.invoke(prompt)
print(result.content)



----- Using Conditional Logic in Prompts -----

However, I would recommend doing some research first. Consider factors like the car's model, year, mileage, and maintenance history. You should also compare prices to ensure you're getting a good deal. If you're unsure, you might want to consult with a mechanic or someone knowledgeable about cars. Lastly, make sure it fits into your budget and long-term financial goals.


In [26]:
# PART 5: Few-Shot Learning Example
print("\n----- Few-Shot Learning Example -----\n")

# Providing a few examples for the model to learn from before asking it to generalize
few_shot_examples = [
    "Translate the following English sentence to French: 'Good morning' -> 'Bonjour'",
    "Translate the following English sentence to French: 'Good night' -> 'Bonne nuit'"
]

# Now, providing a new example to translate
few_shot_prompt = "\n".join(few_shot_examples) + "\nTranslate the following English sentence to French: 'Thank you'"
prompt_template_few_shot = ChatPromptTemplate.from_template(few_shot_prompt)
result = llm.invoke(prompt_template_few_shot)
print(result.content)


----- Few-Shot Learning Example -----



ValueError: Invalid input type <class 'langchain_core.prompts.chat.ChatPromptTemplate'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
# PART 6: Looping Over Multiple Inputs
print("\n----- Looping Over Multiple Inputs -----\n")

template_loop = "Give me advice about {topic}."
prompt_loop = ChatPromptTemplate.from_template(template_loop)

# List of topics
topics = ["time management", "healthy eating", "exercise"]

for topic in topics:
    prompt = prompt_loop.invoke({"topic": topic})
    result = llm.invoke(prompt)
    print(f"Advice on {topic}: {result.content}")



----- Looping Over Multiple Inputs -----

Advice on time management: Sure, I'd be happy to help you with time management! Here are some tips that might be helpful:

1. Prioritize tasks: Make a list of all the tasks you need to accomplish and rank them in order of importance. Focus on completing the most important tasks first.
2. Set goals: Set specific, measurable, achievable, relevant, and time-bound (SMART) goals for yourself. This will help you stay focused and motivated.
3. Use a planner or calendar: Use a planner or calendar to schedule your tasks and appointments. This will help you stay organized and on track.
4. Break tasks into smaller steps: If a task seems overwhelming, break it down into smaller, more manageable steps. This will make it easier to start and complete.
5. Avoid multitasking: Multitasking can be tempting, but it can also lead to mistakes and decreased productivity. Focus on one task at a time.
6. Learn to say no: It's okay to say no to requests that don't alig